# Import Dependencies and Setup Environment

This cell imports all necessary libraries for the LangGraph MCP (Model Context Protocol) agent implementation, including LangChain components, HTTP client libraries, and system utilities required for agent-server communication.

In [1]:
import asyncio
import json
import math
import statistics
import re
import sys
import subprocess
import time
from typing import Any, Dict, List, Optional, Union
from datetime import datetime
from pathlib import Path
import getpass
import os

# HTTP client imports
import requests

# LangChain and LangGraph imports
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# Environment setup
from dotenv import load_dotenv

print("All libraries imported successfully!")

All libraries imported successfully!


# Environment Variable Configuration

This cell configures environment variables by loading from .env file and prompting for Azure OpenAI API key if not already set. Ensures secure credential management for Azure OpenAI service authentication.

In [2]:
# Load environment variables from a .env file if present
load_dotenv()

# Prompt for API key only if not present in environment variables
if not os.environ.get("AZURE_OPENAI_API_KEY"):
    os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter your Azure OpenAI API key: ")

print("API key configured successfully!")

API key configured successfully!


# Azure OpenAI LLM Initialization

This cell initializes the Azure OpenAI chat model with specific configuration parameters including endpoint, API version, model selection (GPT-4o), and temperature settings optimized for consistent tool usage.

In [4]:
# Initialize Azure OpenAI Chat model
llm = AzureChatOpenAI(
    azure_endpoint="https://aoi-ext-eus-aiml-profx-01.openai.azure.com/",
    api_key=os.environ["AZURE_OPENAI_API_KEY"],
    model="gpt-4o",
    api_version="2024-12-01-preview",
    temperature=0.1  # Lower temperature for more consistent tool usage
)

print("Azure OpenAI LLM initialized successfully!")
print(f"Model: gpt-4o")
print(f"Temperature: 0.1 (optimized for tool usage)")

Azure OpenAI LLM initialized successfully!
Model: gpt-4o
Temperature: 0.1 (optimized for tool usage)


# HTTP MCP Client Tools Definition

This cell implements a comprehensive HTTP client interface for Model Context Protocol (MCP) server communication. The implementation includes:

## Architecture Components:
- **HTTP Client Layer**: Uses `requests` library for RESTful API communication with the MCP server
- **LangChain Tool Integration**: Decorates functions with `@tool` decorator for seamless integration with LangGraph agents
- **Error Handling**: Implements robust exception handling with timeout management and status code validation
- **Server Health Monitoring**: Includes health check functionality to verify server availability

## Tool Registration:
All tools are collected into `http_mcp_tools` list for agent registration, enabling dynamic tool discovery and execution within the LangGraph framework.

### Why Local Tool Configuration is Necessary
LangChain/LangGraph Integration: Your agent (running locally) needs to know what tools are available and their schemas. The @tool decorators in your notebook create LangChain tool objects that the agent can reason about and invoke.

**Type Safety & Validation:** The local tool definitions provide:

- Type hints for parameters
- Documentation strings that help the LLM understand what each tool does
- Parameter validation before making HTTP calls

**Abstraction Layer:** The local tools act as a client-side abstraction that:

- Handles HTTP communication details
- Formats requests/responses appropriately
- Provides error handling and timeout management

In [7]:
# HTTP MCP Client Functions and Tools
import requests
from langchain_core.tools import tool

# MCP Server Configuration
MCP_SERVER_BASE_URL = "http://localhost:8000"

def check_mcp_server() -> bool:
    """Check if the MCP HTTP server is running and accessible."""
    try:
        response = requests.get(f"{MCP_SERVER_BASE_URL}/health", timeout=5)
        if response.status_code == 200:
            return True
        else:
            print(f"Server responded with status code: {response.status_code}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"Server not accessible: {e}")
        return False

# Define LangChain tools that connect to HTTP MCP server
@tool
def http_math_statistics(values: list) -> str:
    """
    Calculate comprehensive statistics (mean, median, std dev, etc.) for a dataset.
    
    Args:
        values: List of numerical values
        
    Returns:
        Comprehensive statistical analysis of the dataset
    """
    try:
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/math",
            json={
                "operation": "statistics",
                "values": values
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            stats = result["result"]
            return f"""Statistical Analysis:
                        - Count: {stats['count']}
                        - Mean: {stats['mean']:.4f}
                        - Median: {stats['median']:.4f}
                        - Standard Deviation: {stats['std_dev']:.4f}
                        - Variance: {stats['variance']:.4f}
                        - Min: {stats['min']}
                        - Max: {stats['max']}
                        - Range: {stats['range']}
                        - Sum: {stats['sum']}"""
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error calling math statistics: {str(e)}"

@tool
def http_solve_quadratic(a: float, b: float, c: float) -> str:
    """
    Solve a quadratic equation ax² + bx + c = 0.
    
    Args:
        a: Coefficient of x²
        b: Coefficient of x
        c: Constant term
        
    Returns:
        Solution(s) to the quadratic equation
    """
    try:
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/math",
            json={
                "operation": "quadratic",
                "values": [],
                "a": a,
                "b": b,
                "c": c
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            quad_result = result["result"]
            roots = quad_result["roots"]
            root_type = quad_result["root_type"]
            discriminant = quad_result["discriminant"]
            
            return f"""Quadratic Equation {a}x² + {b}x + {c} = 0:
                        - Discriminant: {discriminant}
                        - Root Type: {root_type}
                        - Solutions: {roots}"""
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error solving quadratic: {str(e)}"

@tool
def http_text_analysis(text: str) -> str:
    """
    Perform comprehensive text analysis including character count, word count, etc.
    
    Args:
        text: The text to analyze
        
    Returns:
        Detailed analysis of the text
    """
    try:
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/text",
            json={
                "operation": "analyze",
                "text": text
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            return f"Text Analysis Results:\n{result['result']}"
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error analyzing text: {str(e)}"

@tool
def http_extract_information(text: str, extraction_type: str) -> str:
    """
    Extract specific information from text (emails, phone numbers, URLs, etc.).
    
    Args:
        text: The text to extract information from
        extraction_type: Type of information to extract (emails, phone_numbers, urls, dates, numbers)
        
    Returns:
        Extracted information of the specified type
    """
    try:
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/text",
            json={
                "operation": "extract",
                "text": text,
                "extraction_type": extraction_type
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            return f"Extracted {extraction_type}:\n{result['result']}"
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error extracting {extraction_type}: {str(e)}"

@tool
def http_transform_text(text: str, transformation: str) -> str:
    """
    Transform text using various transformations (uppercase, lowercase, title_case, pig_latin).
    
    Args:
        text: The text to transform
        transformation: Type of transformation (uppercase, lowercase, title_case, pig_latin)
        
    Returns:
        Transformed text
    """
    try:
        response = requests.post(
            f"{MCP_SERVER_BASE_URL}/text",
            json={
                "operation": "transform",
                "text": text,
                "extraction_type": transformation
            },
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        if result["success"]:
            return f"Transformed text ({transformation}):\n{result['result']}"
        else:
            return f"Error: {result.get('message', 'Unknown error')}"
    except Exception as e:
        return f"Error transforming text: {str(e)}"

# Collect all HTTP MCP tools
http_mcp_tools = [
    http_math_statistics,
    http_solve_quadratic,
    http_text_analysis,
    http_extract_information,
    http_transform_text
]

print("HTTP MCP Client Tools Created Successfully!")
print(f"Available tools: {len(http_mcp_tools)}")
print("Tools can connect to MCP server at:", MCP_SERVER_BASE_URL)

HTTP MCP Client Tools Created Successfully!
Available tools: 5
Tools can connect to MCP server at: http://localhost:8000


# HTTP-based MCP Agent Implementation

This cell defines the core agent creation and streaming functions for HTTP-based MCP communication. Implements ReAct agent pattern with memory persistence and asynchronous streaming capabilities for real-time interaction.

In [8]:
# Create HTTP-based MCP Agent with LangGraph
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

async def create_http_mcp_agent():
    """
    Create a LangGraph ReAct agent using HTTP-based MCP tools.
    
    Returns:
        Configured agent ready for use
    """
    # Check if MCP server is running
    if not check_mcp_server():
        print("ERROR: MCP HTTP server is not running!")
        print("Please start it with: python ../../MCP/mcp-launcher.py start --server http")
        return None
    
    print("MCP HTTP server is accessible")
    
    # Initialize memory for conversation persistence
    memory = MemorySaver()
    
    # Create the ReAct agent with HTTP-based MCP tools
    agent = create_react_agent(
        model=llm,
        tools=http_mcp_tools,
        checkpointer=memory
    )
    
    print("HTTP-based MCP Agent created successfully!")
    print(f"Configured with {len(http_mcp_tools)} tools")
    print("Connected to separately hosted MCP server")
    
    return agent

async def stream_http_agent_updates(user_input: str):
    """
    Stream updates from the HTTP-based MCP agent.
    
    Args:
        user_input (str): The input from the user.
    """
    # Create the agent
    agent = await create_http_mcp_agent()
    if agent is None:
        return
    
    # Configuration for conversation threading
    config = {"configurable": {"thread_id": "http-mcp-demo-thread"}}
    
    print(f"\nProcessing query with HTTP MCP Agent...")
    print(f"Query: {user_input}")
    print("\n" + "="*60)
    
    try:
        # Stream the agent's response
        async for event in agent.astream(
            {"messages": [HumanMessage(content=user_input)]},
            config=config
        ):
            try:
                for value in event.values():
                    if "messages" in value and value["messages"]:
                        last_message = value["messages"][-1]
                        if hasattr(last_message, 'content') and last_message.content:
                            print("Assistant:", last_message.content)
                            print("-" * 40)
            except Exception as e:
                print(f"WARNING: Error processing event: {e}")
                
    except Exception as e:
        print(f"ERROR: Error with HTTP MCP agent: {e}")
        
    print("\nHTTP MCP Agent response completed!")

print("HTTP-based MCP Agent functions ready!")
print("Agent will connect to separately hosted MCP server")
print("Ready to process queries with HTTP communication")

HTTP-based MCP Agent functions ready!
Agent will connect to separately hosted MCP server
Ready to process queries with HTTP communication


# MCP Server Process Management

This cell provides automated server lifecycle management including startup, health monitoring, and graceful shutdown procedures. Handles background process execution and error handling for the HTTP MCP server.

In [9]:
# Auto-start HTTP MCP Server for the demo
import subprocess
import time
import threading
from pathlib import Path

# Global variable to track server process
http_server_process = None

def start_http_mcp_server():
    """Start the HTTP MCP server in the background."""
    global http_server_process
    
    print("Starting HTTP MCP server...")
    
    # Define paths
    mcp_dir = Path("../../MCP").resolve()
    server_script = mcp_dir / "mcp-http-server.py"
    
    if not server_script.exists():
        print(f"ERROR: Server script not found: {server_script}")
        return False
    
    # Check if server is already running
    if check_mcp_server():
        print("SUCCESS: MCP server is already running!")
        return True
    
    try:
        # Start the server in background
        http_server_process = subprocess.Popen(
            [sys.executable, str(server_script)],
            cwd=mcp_dir,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        print(f"INFO: Server starting with PID: {http_server_process.pid}")
        
        # Wait for server to initialize with health checks
        for i in range(15):  # Wait up to 15 seconds
            time.sleep(1)
            print(f"INFO: Waiting for server... ({i+1}/15)")
            
            # Check if process failed
            if http_server_process.poll() is not None:
                stdout, stderr = http_server_process.communicate()
                print(f"ERROR: Server process failed!")
                print(f"stdout: {stdout}")
                print(f"stderr: {stderr}")
                return False
            
            # Check if server is responding to health checks
            if check_mcp_server():
                print("SUCCESS: HTTP MCP server started successfully!")
                print(f"INFO: Server available at: {MCP_SERVER_BASE_URL}")
                print(f"INFO: Health endpoint: {MCP_SERVER_BASE_URL}/health")
                return True
        
        # If we get here, server didn't respond in time
        print("WARNING: Server started but not responding to health checks")
        print("Checking server output...")
        
        # Try to get some output
        try:
            stdout, stderr = http_server_process.communicate(timeout=2)
            if stdout:
                print(f"Server stdout: {stdout[:500]}...")
            if stderr:
                print(f"Server stderr: {stderr[:500]}...")
        except subprocess.TimeoutExpired:
            print("Server process is still running but not responding")
            
        return False
        
    except Exception as e:
        print(f"ERROR: Failed to start server: {e}")
        return False

def stop_http_mcp_server():
    """Stop the HTTP MCP server."""
    global http_server_process
    
    if http_server_process and http_server_process.poll() is None:
        print("INFO: Stopping HTTP MCP server...")
        http_server_process.terminate()
        
        try:
            http_server_process.wait(timeout=5)
            print("SUCCESS: Server stopped gracefully")
        except subprocess.TimeoutExpired:
            http_server_process.kill()
            print("INFO: Server force stopped")
        
        http_server_process = None
    else:
        print("INFO: No server process to stop")

# Start the HTTP MCP server
server_started = start_http_mcp_server()

Starting HTTP MCP server...
Server not accessible: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001F387A363C0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
INFO: Server starting with PID: 36852
INFO: Waiting for server... (1/15)
SUCCESS: HTTP MCP server started successfully!
INFO: Server available at: http://localhost:8000
INFO: Health endpoint: http://localhost:8000/health


# Mathematical Operations Testing

This cell demonstrates the agent's mathematical capabilities by testing statistical analysis functions and quadratic equation solving through the HTTP MCP server. Validates tool integration and numerical computation accuracy.

In [10]:
# Test 1: Mathematical Operations with HTTP MCP Agent
print("Testing Mathematical Operations via HTTP MCP Server")
print("=" * 60)

math_query = """
I have a dataset with the following values: [85, 92, 78, 96, 89, 91, 87, 94, 82, 88].
Please calculate the mean, median, and standard deviation for this dataset.
Also, solve the quadratic equation 2x² - 7x + 3 = 0.
Finally, analyze the distribution characteristics of the data.
"""

print(f"Query: {math_query}")
print("\nAgent Response (via HTTP MCP Server):")
print("=" * 60)

# Run the query using the HTTP-based MCP agent
await stream_http_agent_updates(math_query)

Testing Mathematical Operations via HTTP MCP Server
Query: 
I have a dataset with the following values: [85, 92, 78, 96, 89, 91, 87, 94, 82, 88].
Please calculate the mean, median, and standard deviation for this dataset.
Also, solve the quadratic equation 2x² - 7x + 3 = 0.
Finally, analyze the distribution characteristics of the data.


Agent Response (via HTTP MCP Server):
MCP HTTP server is accessible
HTTP-based MCP Agent created successfully!
Configured with 5 tools
Connected to separately hosted MCP server

Processing query with HTTP MCP Agent...
Query: 
I have a dataset with the following values: [85, 92, 78, 96, 89, 91, 87, 94, 82, 88].
Please calculate the mean, median, and standard deviation for this dataset.
Also, solve the quadratic equation 2x² - 7x + 3 = 0.
Finally, analyze the distribution characteristics of the data.


Assistant: Quadratic Equation 2.0x² + -7.0x + 3.0 = 0:
                        - Discriminant: 25.0
                        - Root Type: real_distinct
   

# Text Processing and Information Extraction Testing

This cell tests the agent's natural language processing capabilities including text analysis, pattern extraction (emails, phone numbers), and text transformation. Demonstrates multi-modal tool usage and information extraction workflows.

In [11]:
# Test 2: Text Processing with HTTP MCP Agent
print("\n" + "="*60)
print("Testing Text Processing via HTTP MCP Server")
print("=" * 60)

text_query = """
Please analyze the following text and extract useful information:

"Artificial Intelligence and Machine Learning are revolutionizing technology. 
Our research team has achieved 95.3% accuracy in natural language processing tasks.
Contact us at research@aicompany.com or call (555) 123-4567 for collaboration.
Visit our website at https://ai-revolution.com to learn more about our work!"

I need:
1. A comprehensive text analysis with word count, character analysis, etc.
2. Extract all email addresses and phone numbers from the text
3. Transform the first sentence to pig latin
4. Provide insights about the text structure and content
"""

print(f"Query: {text_query}")
print("\nAgent Response (via HTTP MCP Server):")
print("=" * 60)

# Run the query using the HTTP-based MCP agent
await stream_http_agent_updates(text_query)


Testing Text Processing via HTTP MCP Server
Query: 
Please analyze the following text and extract useful information:

"Artificial Intelligence and Machine Learning are revolutionizing technology. 
Our research team has achieved 95.3% accuracy in natural language processing tasks.
Contact us at research@aicompany.com or call (555) 123-4567 for collaboration.
Visit our website at https://ai-revolution.com to learn more about our work!"

I need:
1. A comprehensive text analysis with word count, character analysis, etc.
2. Extract all email addresses and phone numbers from the text
3. Transform the first sentence to pig latin
4. Provide insights about the text structure and content


Agent Response (via HTTP MCP Server):
MCP HTTP server is accessible
HTTP-based MCP Agent created successfully!
Configured with 5 tools
Connected to separately hosted MCP server

Processing query with HTTP MCP Agent...
Query: 
Please analyze the following text and extract useful information:

"Artificial Inte